# HTML解析入门及准备URL生成连续技
![for humans](https://requests-html.kennethreitz.org/_static/requests-html-logo.png#thumbnail)

*  本周主要内容：HTML解析（parse HTML）及准备URL生成连续技
*  上周主要内容：HTML解析（parse HTML）及Xpath实践
*  20春_Web数据挖掘_week03
*  电子讲义设计者：廖汉腾, 许智超
<br/>
<br/>

-----
## 复习

复习：上周内容，实践

* HTML解析（parse HTML）: requests-html  丶
* Xpath实践
* m.liepin.com 取工作牛肉

-----
## 本周内容及学习目标

本周内容聚焦在

<mark> 如何从一页开始有系统的找更多页的内容 </mark>

为此，我们需要学习

1. 拆解带有参数的URL，并再从query取出参数
   a. URL拆解: 使用 urllib.parse 解析 出query
   b. query拆解:  取出参数 成python字典
2. 有基底URL，加上参数字典，请求新网页连续技

我们除了继续学习解决上一周已开始面对的以下挑战：
![Xpath Axis](http://krum.rz.uni-mannheim.de/inet-2005/images/xpath-axis.gif)

### 旧目标
1. 使用 requests-html 爬取并存取网页文字档，查找[requests-html 中文文档](https://cncert.github.io/requests-html-doc-cn/#/)
2. 熟悉 [xpath 语法](https://www.w3cschool.cn/xpath/xpath-syntax.html)丶[xpath 节点](https://www.w3cschool.cn/xpath/xpath-nodes.html)
3. 使用 [xpath cheatsheet](https://devhints.io/xpath)
  * 在 Chrome Inspector 使用
  * 在 requests-html (Python) 使用
4. 简易使用 [pd.DataFrame](https://www.pypandas.cn/doc/getting_started/dsintro.html#dataframe)

### 新目标
这一周，学生将实践
* 猎聘PC版 liepin.com 取工作URL参数的牛肉
* 如何生成一连串新URL以进一步爬取数据




In [4]:
%%html
<style>
/* 本电子讲义使用之CSS */
div.code_cell {
    background-color: #e5f1fe;
}
div.cell.selected {
    background-color: #effee2;
    font-size: 2rem;
    line-height: 2.4rem;
}
div.cell.selected .rendered_html table {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html pre code {
    background-color: #C4E4ff;   
    padding: 2px 25px;
}
.rendered_html pre {
    background-color: #99c9ff;
}
div.code_cell .CodeMirror {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html img, .rendered_html svg {
    max-width: 60%;
    height: auto;
    float: right;
}

.rendered_html img[src*="#full"], .rendered_html svg[src*="#full"] {
    max-width: 100%;
    height: auto;
    float: none;
}

.rendered_html img[src*="#thumbnail"], .rendered_html svg[src*="#thumbnail"] {
    max-width: 15%;
    height: auto;
}

/* Gradient transparent - color - transparent */
hr {
    border: 0;
    border-bottom: 1px dashed #ccc;
}
.emoticon{
    font-size: 5rem;
    line-height: 4.4rem;
    text-align: center;
    vertical-align: middle;
}
.bg-split_apply_comine {
    width: 500px;     
    height: 300px;
    background: url('02_split-apply-comine_500x300.png') -10px -10px;
    float: right;
}
.bg-comine {
    width: 175px;
    height: 150px;
    background: url('02_split-apply-comine_500x300.png') -280px -80px;
    float: right;
}
.bg-apply {
    width: 155px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -160px -30px;
    float: right;
}
.bg-split {
    width: 205px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -10px -30px;
    float: right;
}
.break {
                   page-break-after: right; 
                   width:700px;
                   clear:both;
}
</style>

In [5]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession

# 0. 上周加分作业解答

In [6]:
# C-1   单一页面
url = "https://m.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

# C-5
# 难: '公司URL', '时间', '经验'

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//div[@class="job-card-wrap"]//div[@class="job-card"]')

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        '经验':      './/ul/li[time]/text()'
    },
    'text_content': {
        '职称':    './/ul/li/a[contains(@class,"job-name")]/span[@class="name-text"]', 
        '薪水':    './/ul/li/a[contains(@class,"job-name")]/following-sibling::span', 
        '公司地点':'.//ul/li/time/following-sibling::a',
        '公司名称': './/ul/li/a[contains(@class,"company-name")]', 
        '时间':    './/ul/li/time', 
    },
    'href': {
        '链结':    './/ul/li/a[contains(@class,"job-name")]', 
        '公司URL': './/ul/li/a[contains(@class,"company-name")]', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

print ([len(v) for k,v in 数据字典.items()])  # 檢查

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week02_liepin.xlsx", sheet_name="搜查结果")
数据 

[1, 1, 1, 1, 1, 1, 1, 1]


,职称,薪水,公司地点,公司名称,时间,经验,链结,公司URL
0,产品经理,15-20k·12薪,肇庆,广东国腾量子科技有限公司,2020-04-01,1年以上 硕士及以上,https://m.liepin.com/job/1925551201.shtml,https://m.liepin.com/company/9831379/


#### 小坑/小风格
* 代码某几行最后一个字符有 \，指的是什麽意思？
* 代码某几行最后一个字符有 \，为什麽要用？给机器还是人用的？
* 代码某几行最后一个字符有 \，若后面多了空白会怎麽样？

----
答案: 和机器说，此行代码未结束，下行继续，最主要是让**写**程序的人可以合法回车，目标主要是为了让**读**代码的人可以因为好的回车排版，更易懂代码的意义


-----

# 本周目标
* [猎聘PC版](https://www.liepin.com/zhaopin/)
* 上方导航有  公司行业 城市 薪资 的分页选单
* 请练习xpath抽出数据

## Xpath解析HTML

In [7]:
# A-1   单一页面
url = "https://www.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//ul[@class="sojob-list"]/li')

# 预期是一个元素的列表？
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]'))
#print (主要元素[0].xpath('//div[contains(@class,"job-info")]/h3/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="text-warning"]'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/time/@title'))
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a'))

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
数据 

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL
0,大专及以上,2年以上,6-8k·12薪,2020年04月06日,大客户主管、专员,武汉,武汉复星汉正街房地产开发有限公司,https://www.liepin.com/job/1927240251.shtml,https://www.liepin.com/company/8549940/
1,统招本科,3年以上,面议,2020年04月06日,人力资源主管,青岛,青岛碧桂园产城发展有限公司,https://www.liepin.com/job/1927239763.shtml,https://www.liepin.com/company/9654764/
2,本科及以上,5年以上,20-30k·12薪,2020年04月06日,云计算销售经理,上海-虹桥,富通东方,https://www.liepin.com/job/1927239637.shtml,https://www.liepin.com/company/895509/
3,统招本科,3年以上,面议,2020年04月06日,品牌媒介主管,青岛-中韩,青岛碧桂园产城发展有限公司,https://www.liepin.com/job/1927239617.shtml,https://www.liepin.com/company/9654764/
4,统招本科,5年以上,15-20k·12薪,2020年04月06日,口腔医生/儿牙医生,济南,济南可恩口腔医院有限公司,https://www.liepin.com/job/1927239607.shtml,https://www.liepin.com/company/8267148/
5,大专及以上,1年以上,10-20k·12薪,2020年04月06日,医疗美容整形材料销售,广州,意之美(广州)生物科技有限公司,https://www.liepin.com/job/1927224799.shtml,https://www.liepin.com/company/12001657/
6,大专及以上,1年以上,10-20k·12薪,2020年04月06日,医疗美容整形材料销售,深圳,意之美(广州)生物科技有限公司,https://www.liepin.com/job/1927224787.shtml,https://www.liepin.com/company/12001657/
7,大专及以上,3年以上,5-10k·12薪,2020年04月06日,渠道推广（陕南陕北）,西安-张家堡,陕西长宇酒业有限公司,https://www.liepin.com/job/1927215425.shtml,https://www.liepin.com/company/9624258/
8,大专及以上,4年以上,15-35k·13薪,2020年04月06日,产品经理,深圳-南油,上海隆长信息技术有限公司,https://www.liepin.com/job/1927208933.shtml,https://www.liepin.com/company/9707913/
9,本科及以上,5年以上,33-60k·12薪,2020年04月06日,投资合作总经理/总监（投合项目）,重庆-渝北区,重庆海成实业(集团)有限公司,https://www.liepin.com/job/1927175665.shtml,https://www.liepin.com/company/7902708/


In [8]:
# A-2 扩张 公司 ?  

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath('//div[@data-selector="search-conditions"]')
# 预期是一个元素的列表？
print (主要元素)
print (主要元素[0])
print (主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print (x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print (x)  
    

公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a/@href')
               
公司数据选择器链结

# 但我们需要知道这些选择器链结, 对映到什麽数据
公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a')
公司数据选择器链结

#[ x.xpath("a/@href")[0] for x in 公司数据选择器链结]
#[ x.xpath("a/text()")[0] for x in 公司数据选择器链结]
公司数据选择器链结 = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 公司数据选择器链结}
公司数据选择器链结

[<Element 'div' class=('search-conditions',) data-selector='search-conditions'>]
<Element 'div' class=('search-conditions',) data-selector='search-conditions'>
[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'中国500强': '/zhaopin/?init=-1&headckid=00fb5387eeae0ac8&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=00fb5387eeae0ac8&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6cc72f710c7ec32baa27bf7f4f986c23&d_curPage=0&d_pageSize=40&d_headId=6cc72f710c7ec32baa27bf7f4f986c23',
 '2018互联网300强': '/zhaopin/?init=-1&headckid=00fb5387eeae0ac8&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=00fb5387eeae0ac8&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6cc72f710c7ec32baa27bf7f4f986c23&d_curPage=0&d_pageSize=40&d_headId=6cc72f710c7ec32baa27bf7f4f986c23',
 '制造业500强': '/zhaopin/?init=-1&headckid=00fb5387eeae0ac8&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=00fb5387eeae0ac8&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6cc72f710c7ec32baa27bf7f4f986c23&d_curPage=0&d_pageSize=40&d_headId=6cc72f710c7ec32baa27bf7f4f986c23',
 'AI创新成长50强 ': '/zhaopin/?init=-1&headckid=

## 使用urllib3 解析 url 
上面的url应该触动不同的页面查询，但能不能轻松无误的拆分url并进行比较？

### urllib模块功能介绍
* urlparse 
返回的6个部分，分别是：scheme(机制)丶netloc(网络位置)丶path(路径)丶params(路径段参数)丶query(查询)丶fragment(片段)。
* parse_qs
返回query(查询)多个部分

In [9]:
# B-1 使用 urllib.parse 解析
from urllib.parse import urlparse, parse_qs
[ urlparse(x) for x in 公司数据选择器链结.values()]

[ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=00fb5387eeae0ac8&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=00fb5387eeae0ac8&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6cc72f710c7ec32baa27bf7f4f986c23&d_curPage=0&d_pageSize=40&d_headId=6cc72f710c7ec32baa27bf7f4f986c23', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=00fb5387eeae0ac8&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=00fb5387eeae0ac8&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6cc72f710c7ec32baa27bf7f4f986c23&d_curPage=0&d_pageSize=40&d_headId=6cc72f710c7ec32baa27bf7f4f986c23', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=00fb5387eeae0ac8&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=00fb5387eeae0ac8&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom

In [10]:
# B-2 使用 pd.DataFrame进行 unuinque()相异值计量比对 
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 公司数据选择器链结.values()])
df.info()
print(df.nunique())
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    6 non-null      object
 1   netloc    6 non-null      object
 2   path      6 non-null      object
 3   params    6 non-null      object
 4   query     6 non-null      object
 5   fragment  6 non-null      object
dtypes: object(6)
memory usage: 416.0+ bytes
scheme      1
netloc      1
path        1
params      1
query       6
fragment    1
dtype: int64


,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=00fb5387eeae0ac8&flushckid=1&...,


In [11]:
# B-3 针对query 再解析之 
#df_qs = pd.DataFrame([ parse_qs(x) for x in df['query'] ])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','compTag']]

init             1
headckid         1
flushckid        1
fromSearchBtn    1
keyword          1
compTag          6
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


,keyword,compTag
0,PRD,155
1,PRD,182
2,PRD,186
3,PRD,189
4,PRD,130
5,PRD,156


In [12]:
# B-3-X 对 B-3 代码的字典/列表推导的分拆说明

# ----------------------------------------------
# 列表暂存 = [{k:v[0] for k,v in parse_qs(q).items()} for q in df['query'] ]
# 以下3行代碼相當於上面推導1行
列表暂存 = [] # list()
for x in df['query']: 
    列表暂存.append({k:v[0] for k,v in parse_qs(x).items()} )
    
# ----------------------------------------------
# 字典暂存 = {k:v[0] for k,v in parse_qs(x).items()}
# 以下3行代碼相當於上面推導1行
字典暂存 = dict()
for k,v in parse_qs(x).items():           # for 键,值 in 字典.items():
    字典暂存.update({k:v[0]})
    

# ----------------------------------------------
print (列表暂存, 字典暂存) 

'''python 原代碼以下不處理
df_qs = pd.DataFrame(list_query)
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','compTag']]
'''

[{'init': '-1', 'headckid': '00fb5387eeae0ac8', 'flushckid': '1', 'fromSearchBtn': '2', 'keyword': 'PRD', 'compTag': '155', 'ckid': '00fb5387eeae0ac8', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '6cc72f710c7ec32baa27bf7f4f986c23', 'd_curPage': '0', 'd_pageSize': '40', 'd_headId': '6cc72f710c7ec32baa27bf7f4f986c23'}, {'init': '-1', 'headckid': '00fb5387eeae0ac8', 'flushckid': '1', 'fromSearchBtn': '2', 'keyword': 'PRD', 'compTag': '182', 'ckid': '00fb5387eeae0ac8', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '6cc72f710c7ec32baa27bf7f4f986c23', 'd_curPage': '0', 'd_pageSize': '40', 'd_headId': '6cc72f710c7ec32baa27bf7f4f986c23'}, {'init': '-1', 'headckid': '00fb5387eeae0ac8', 'flushckid': '1', 'fromSearchBtn': '2', 'keyword': 'PRD', 'compTag': '186', 'ckid': '00fb5387eeae0ac8', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '6cc72f71

"python 原代碼以下不處理\ndf_qs = pd.DataFrame(list_query)\nprint (df_qs.nunique())\ndf_qs.head()\ndf_qs[['keyword','compTag']]\n"

In [13]:
# B-3-XX 整合后

列表暫存 = [] # list()
for q in df['query']: 
    字典暫存 = dict()
    for k,v in parse_qs(q).items(): # for 鍵,值 in 字典.items():
        字典暫存.update({k:v[0]})
    列表暫存.append(字典暫存)
列表暫存
# ----------------------------------------------
# 比較
# 列表暫存 = [{k:v[0] for k,v in parse_qs(q).items()} for q in df['query'] ]

[{'init': '-1',
  'headckid': '00fb5387eeae0ac8',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': 'PRD',
  'compTag': '155',
  'ckid': '00fb5387eeae0ac8',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw',
  'd_sfrom': 'search_unknown',
  'd_ckId': '6cc72f710c7ec32baa27bf7f4f986c23',
  'd_curPage': '0',
  'd_pageSize': '40',
  'd_headId': '6cc72f710c7ec32baa27bf7f4f986c23'},
 {'init': '-1',
  'headckid': '00fb5387eeae0ac8',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': 'PRD',
  'compTag': '182',
  'ckid': '00fb5387eeae0ac8',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw',
  'd_sfrom': 'search_unknown',
  'd_ckId': '6cc72f710c7ec32baa27bf7f4f986c23',
  'd_curPage': '0',
  'd_pageSize': '40',
  'd_headId': '6cc72f710c7ec32baa27bf7f4f986c23'},
 {'init': '-1',
  'headckid': '00fb5387eeae0ac8',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': 'PRD',
  'compTag': '186',
  'ckid': '00fb5387eeae0ac8',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc

### 小结
* comTag 是不同的公司选择器, 数值不样, 对映到不同类型的公司
* keyword 是搜查关键字

In [14]:
# B-4 建构 参数模板 及 字典_compTag
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])
print(参数模板)
# [ parse_url_qs_for_compTag(x)['compTag'] for x in 公司数据选择器链结.values()]
[ parse_url_qs_for_compTag(x)['compTag'][0] for x in 公司数据选择器链结.values()]

字典_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in 公司数据选择器链结.items()}
print (字典_compTag)


{'init': ['-1'], 'headckid': ['00fb5387eeae0ac8'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['PRD'], 'compTag': ['155'], 'ckid': ['00fb5387eeae0ac8'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6cc72f710c7ec32baa27bf7f4f986c23'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6cc72f710c7ec32baa27bf7f4f986c23']}
{'中国500强': '155', '2018互联网300强': '182', '制造业500强': '186', 'AI创新成长50强 ': '189', '独角兽': '130', '上市公司': '156'}


In [15]:
# B-5 建构 参数模板  
def 参数模板生成(compTag , keyword ):
    参数 = 参数模板.copy()
    参数['compTag'] = compTag
    参数['keyword'] = keyword
    return (参数)

参数_compTag_用户体验 = { k:参数模板生成(compTag = [v], keyword = ['用户体验']) for k,v in 字典_compTag.items()}
print(参数_compTag_用户体验)

{'中国500强': {'init': ['-1'], 'headckid': ['00fb5387eeae0ac8'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['155'], 'ckid': ['00fb5387eeae0ac8'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6cc72f710c7ec32baa27bf7f4f986c23'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6cc72f710c7ec32baa27bf7f4f986c23']}, '2018互联网300强': {'init': ['-1'], 'headckid': ['00fb5387eeae0ac8'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['182'], 'ckid': ['00fb5387eeae0ac8'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6cc72f710c7ec32baa27bf7f4f986c23'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6cc72f710c7ec32baa27bf7f4f986c23']}, '制造业500强': {'init': ['-1'], 'headckid': ['00fb5387eeae0ac8'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['186'], 'ckid': ['00fb5387eeae0ac8'], '

## requests 生成

In [16]:
# C-1   多个页面准备测试1 中国500强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验['中国500强']
r = session.get( url, params = payload)
r.url

'https://www.liepin.com/zhaopin/?init=-1&headckid=00fb5387eeae0ac8&flushckid=1&fromSearchBtn=2&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&compTag=155&ckid=00fb5387eeae0ac8&siTag=1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6cc72f710c7ec32baa27bf7f4f986c23&d_curPage=0&d_pageSize=40&d_headId=6cc72f710c7ec32baa27bf7f4f986c23'

In [17]:
# C-2  简化 A-1   单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)




In [18]:
# C-3   多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin( url, params = payload)
    df = df.assign (热门公司类型 = k)    
    list_df.append(df)

df_all = pd.concat(list_df)
df_all

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL,热门公司类型
0,本科及以上,1年以上,8-12k·13薪,2020年04月05日,HRBP助理,广州-天河区,网易集团,https://www.liepin.com/job/1927237301.shtml,https://www.liepin.com/company/5964833/,中国500强
1,本科及以上,3年以上,10-20k·16薪,2020年04月05日,大客户销售经理-杭州-网易严选,杭州,网易集团,https://www.liepin.com/job/1926482689.shtml,https://www.liepin.com/company/5964833/,中国500强
2,统招本科,5年以上,面议,2020年04月05日,家具/家居产品设计师-严选事业部,杭州-滨江区,网易集团,https://www.liepin.com/job/1926109575.shtml,https://www.liepin.com/company/5964833/,中国500强
3,大专及以上,2年以上,面议,2020年04月03日,广告销售大客户经理,,网易集团,https://www.liepin.com/job/1919211941.shtml,https://www.liepin.com/company/5964833/,中国500强
4,统招本科,10年以上,面议,2020年04月02日,集团办公室副主任,苏州-吴江区,盛虹控股集团有限公司,https://www.liepin.com/job/1927078451.shtml,https://www.liepin.com/company/7966465/,中国500强
...,...,...,...,...,...,...,...,...,...,...
35,本科及以上,5年以上,15-25k·12薪,2020年03月31日,市政配套高级专业经理,北京-马坡,新城控股集团住宅开发事业部,https://www.liepin.com/job/1916401169.shtml,https://www.liepin.com/company/442197/,上市公司
36,统招本科,经验不限,10-15k·12薪,2020年03月30日,出国考试教师-雅思,北京,新东方教育科技集团有限公司,https://www.liepin.com/job/1922163241.shtml,https://www.liepin.com/company/2485876/,上市公司
37,本科及以上,3年以上,10-15k·12薪,2020年03月29日,佛山分行私行综合管理岗,佛山-乐从,平安银行,https://www.liepin.com/job/1927053295.shtml,https://www.liepin.com/company/7954197/,上市公司
38,本科及以上,5年以上,25-50k·12薪,2020年03月29日,支行行长,佛山-乐从,平安银行,https://www.liepin.com/job/1918705599.shtml,https://www.liepin.com/company/7954197/,上市公司


In [19]:
# C-4   输出
df_all.to_excel("20春_Web数据挖掘_week03_liepin_各热门公司类型.xlsx", sheet_name="搜查结果")

In [20]:
# C-5 Pandas  基本能力

print (df_all.nunique())
df_all[['edu']].drop_duplicates()

df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

edu         6
经验         10
薪水         72
时间         22
职称        172
公司地点       80
公司名称       64
链结        188
公司URL      64
热门公司类型      6
dtype: int64


职称
公司名称         edu      
天能电池集团股份有限公司 统招本科   16
华为           本科及以上  12
             统招本科   10
天能电池集团股份有限公司 本科及以上   8
             大专及以上   8
...                 ..
深圳市豪鹏科技有限公司  本科及以上   1
玖龙纸业(控股)有限公司 本科及以上   1
百济神州         学历不限    1
中环环保         大专及以上   1
江苏南通二建集团有限公司 大专及以上   1

[89 rows x 1 columns]

In [21]:
df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

职称
公司名称         edu      
天能电池集团股份有限公司 统招本科   16
华为           本科及以上  12
             统招本科   10
天能电池集团股份有限公司 本科及以上   8
             大专及以上   8
...                 ..
深圳市豪鹏科技有限公司  本科及以上   1
玖龙纸业(控股)有限公司 本科及以上   1
百济神州         学历不限    1
中环环保         大专及以上   1
江苏南通二建集团有限公司 大专及以上   1

[89 rows x 1 columns]

In [22]:
df_all.groupby(['公司名称','edu']).agg({"薪水":"count"}).sort_values(by='薪水', ascending=False)

薪水
公司名称         edu      
天能电池集团股份有限公司 统招本科   16
华为           本科及以上  12
             统招本科   10
天能电池集团股份有限公司 本科及以上   8
             大专及以上   8
...                 ..
深圳市豪鹏科技有限公司  本科及以上   1
玖龙纸业(控股)有限公司 本科及以上   1
百济神州         学历不限    1
中环环保         大专及以上   1
江苏南通二建集团有限公司 大专及以上   1

[89 rows x 1 columns]

In [23]:
df_all.groupby(['公司名称','edu']).agg({"经验":"count"}).sort_values(by='经验', ascending=False)

经验
公司名称         edu      
天能电池集团股份有限公司 统招本科   16
华为           本科及以上  12
             统招本科   10
天能电池集团股份有限公司 本科及以上   8
             大专及以上   8
...                 ..
深圳市豪鹏科技有限公司  本科及以上   1
玖龙纸业(控股)有限公司 本科及以上   1
百济神州         学历不限    1
中环环保         大专及以上   1
江苏南通二建集团有限公司 大专及以上   1

[89 rows x 1 columns]

# 本周练习

一样反向工程解析:

## 上方界面的params参数
* 公司：v
* 行业：?
* 城市：?
* 薪资：?
## 下方界面的params参数
* 跳转到 N 页确定 ?
## 换  
* keyword


In [31]:
# 换  keyword
url = "https://www.liepin.com/zhaopin/?keyword=UX"
session = HTMLSession()
r = session.get( url )

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//ul[@class="sojob-list"]/li')

# 预期是一个元素的列表？
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]'))
#print (主要元素[0].xpath('//div[contains(@class,"job-info")]/h3/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="text-warning"]'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/time/@title'))
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a'))

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week03_UX.xlsx", sheet_name="搜查结果")
数据 

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL
0,大专及以上,2年以上,6-8k·12薪,2020年04月06日,大客户主管、专员,武汉,武汉复星汉正街房地产开发有限公司,https://www.liepin.com/job/1927240251.shtml,https://www.liepin.com/company/8549940/
1,统招本科,3年以上,面议,2020年04月06日,人力资源主管,青岛,青岛碧桂园产城发展有限公司,https://www.liepin.com/job/1927239763.shtml,https://www.liepin.com/company/9654764/
2,本科及以上,5年以上,20-30k·12薪,2020年04月06日,云计算销售经理,上海-虹桥,富通东方,https://www.liepin.com/job/1927239637.shtml,https://www.liepin.com/company/895509/
3,统招本科,3年以上,面议,2020年04月06日,品牌媒介主管,青岛-中韩,青岛碧桂园产城发展有限公司,https://www.liepin.com/job/1927239617.shtml,https://www.liepin.com/company/9654764/
4,统招本科,5年以上,15-20k·12薪,2020年04月06日,口腔医生/儿牙医生,济南,济南可恩口腔医院有限公司,https://www.liepin.com/job/1927239607.shtml,https://www.liepin.com/company/8267148/
5,大专及以上,1年以上,10-20k·12薪,2020年04月06日,医疗美容整形材料销售,广州,意之美(广州)生物科技有限公司,https://www.liepin.com/job/1927224799.shtml,https://www.liepin.com/company/12001657/
6,大专及以上,1年以上,10-20k·12薪,2020年04月06日,医疗美容整形材料销售,深圳,意之美(广州)生物科技有限公司,https://www.liepin.com/job/1927224787.shtml,https://www.liepin.com/company/12001657/
7,大专及以上,3年以上,5-10k·12薪,2020年04月06日,渠道推广（陕南陕北）,西安-张家堡,陕西长宇酒业有限公司,https://www.liepin.com/job/1927215425.shtml,https://www.liepin.com/company/9624258/
8,大专及以上,4年以上,15-35k·13薪,2020年04月06日,产品经理,深圳-南油,上海隆长信息技术有限公司,https://www.liepin.com/job/1927208933.shtml,https://www.liepin.com/company/9707913/
9,本科及以上,5年以上,33-60k·12薪,2020年04月06日,投资合作总经理/总监（投合项目）,重庆-渝北区,重庆海成实业(集团)有限公司,https://www.liepin.com/job/1927175665.shtml,https://www.liepin.com/company/7902708/
